In [ ]:
import numpy as np

In [ ]:
neurons = [4,4,3]

In [ ]:
weights = None
filename = "weights.npy"
try:
    weights = np.load(filename)
except:
    weights = np.zeros((len(neurons)-1, max(neurons), max(neurons)+1))
    for i in range(len(weights)):
        layer = np.random.rand(neurons[i+1], neurons[i]+1)
        layer[:,0] = np.random.rand() # bias for that layer
        layer.resize(max(neurons), max(neurons)+1)
        weights[i] = layer
weights

In [ ]:
from sklearn import datasets, preprocessing
iris = datasets.load_iris()
data = preprocessing.normalize(iris.data)
iris.target

In [ ]:
# run inference
def inference(weights, data):
    def activation(x):
        return (1 / (1+np.pow(np.e, -x)))

    sample = data
    sample.resize(max(neurons))

    # add 1 at the beginning so bias properly applies 
    sample = np.pad(sample, (1,0), "constant", constant_values=(1))
    for (i,layer) in enumerate(weights):
        sample = layer @ sample
        sample = activation(sample)
        sample = np.pad(sample, (1,0), "constant", constant_values=(1))
    return sample[1:(neurons[-1]+1)] # strip all the useless padding

In [ ]:
# test error
for (sample, target) in zip(data, iris.target):
    targetArr = np.zeros(3)
    targetArr[target] = 1
    output = inference(weights, sample)
    err = np.sum(np.pow(targetArr - output, 2))/2
    print(err)


In [ ]:
np.save(filename, weights)